In [1]:
import pandas as pd
import xlwings as xw
from scipy.optimize import brentq
from numpy import exp, log
from typing import Tuple, List

def npv(years: pd.Series, cashflows: pd.Series, rate: float) -> float:
    discount_factors = exp(-rate * years)
    return (cashflows * discount_factors).sum()

def get_yield(rate:float , years: int) -> float:
    return exp(rate * years) - 1

def simple_irr_solve(years: int, start_flow: int, end_flow: int) -> float:
    return log(end_flow / start_flow) / years

def solve_irr(years: pd.Series, cashflows: pd.Series) -> float:
    if (cashflows != 0).sum() == 2:
        return simple_irr_solve(years.iloc[-1], cashflows.iloc[0], cashflows.iloc[-1])
    try:
        irr = brentq(lambda r: npv(years, cashflows, r), -1, 1)
    except ValueError:
        irr = brentq(lambda r: npv(years, cashflows, r), -1000, 1000)

    if abs(npv(years, cashflows, irr)) > 1000: 
        raise ValueError(f'IRR calculation failed for cashflows {cashflows} and years {years}: {irr}')
    return irr

def get_effective_cashflow(cashflow: pd.Series, pnl: pd.Series) -> Tuple[pd.Series, pd.Series]:
    eff = cashflow * -1.0
    eff.iloc[[0, -1]] = -pnl.iloc[0], eff.iloc[-1] + pnl.iloc[-1]
    days = eff.index.to_series().diff().dt.years.cumsum().fillna(0)

    return days, eff


In [2]:
years = pd.Series([0, 1, 2, 3, 4, 5])
cashflows = pd.Series([-100, 20, 20, 20, 20, 120])
solve_irr(years, cashflows)

0.18232155679395576

In [13]:
import xlwings as xw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the notebook for inline plotting
%matplotlib inline

# Set visual style
sns.set(style="whitegrid")

# Load data
def load_data(file_name, sheet_name):
    wb = xw.Book(file_name)
    sheet = wb.sheets[sheet_name]
    return sheet.range('A1').options(pd.DataFrame, expand='table', index=False).value

data = load_data('Ledger.xlsm', 'data')



In [15]:
data

,Fecha,Titular,Cuenta,Linea,Activos CLP,Activos USD,Pasivos CLP,Pasivos USD,CLP,USD,...,VM,Unexplained,Comentarios,Investment Names,Investment Accounts,Investment Lines,Net Worth CLP,Net Worth USD,Total Expenses,Total Income
0,2024-01-09,M.C.C.,Santander,CC,812000.0,0.00,0.0,0.0,812000.0,0.00,...,NaN,NaN,None,Fintual-MM,Fintual,MM,812000.0,0.00,-500630.92,0.0
1,2024-01-09,M.C.C.,Santander,TC 5252,9460608.0,6952.09,-10000000.0,-8000.0,-539392.0,-1047.91,...,NaN,NaN,None,Fintual-A,Fintual,A,19460608.0,14952.09,0.00,0.0
2,2024-01-09,M.C.C.,Fintual,MM,4852676.0,0.00,0.0,0.0,4852676.0,0.00,...,NaN,NaN,None,Fintual-APV-B,Fintual,APV-B,4852676.0,0.00,0.00,0.0
3,2024-01-09,M.C.C.,Fintual,A,435540.0,0.00,0.0,0.0,435540.0,0.00,...,NaN,NaN,None,Fintual-ACWI,Fintual,ACWI,435540.0,0.00,0.00,0.0
4,2024-01-09,M.C.C.,Fintual,APV-B,210492.0,0.00,0.0,0.0,210492.0,0.00,...,NaN,NaN,None,Fintual-MCHI,Fintual,MCHI,210492.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,2024-06-27,M.C.C.,Racional,RDDT,0.0,387.16,0.0,0.0,0.0,387.16,...,3.514961e+04,0.0,None,None,None,None,0.0,387.16,0.00,0.0
603,2024-06-27,M.C.C.,Racional,LTM,0.0,0.00,0.0,0.0,0.0,0.00,...,0.000000e+00,0.0,None,None,None,None,0.0,0.00,0.00,0.0
604,2024-06-27,M.C.C.,Binance,Crypto,0.0,939.28,0.0,0.0,0.0,939.28,...,-3.637979e-12,0.0,None,None,None,None,0.0,939.28,0.00,0.0
605,2024-06-27,M.C.C.,World-Coin,WC,0.0,376.54,0.0,0.0,0.0,376.54,...,6.077876e+03,0.0,None,None,None,None,0.0,376.54,0.00,0.0
